In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path

#instead of using namedtuple approach, we can use below approach of class variables
#advantage of this approach is here we can define datatype of each variable.
@dataclass(frozen=True)
class PrepareCallbacksConfig:
  root_dir: Path
  tensorboard_root_log_dir: Path
  checkpoint_model_filepath: Path

In [3]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath) #reading config file
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig: #o/p datatype will be PrepareCallbacksConfig
        config = self.config.prepare_callbacks #passing data to local config variable. This data prepare_callbacks comes from config.yaml file
        
        #creating directory not only for root but for checkpoint and tensorboard logs.
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath) #dirname gives dir filename/filepath for model checkpoint. This data comes from config.yaml file
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ]) 

        prepare_callbacks_config = PrepareCallbacksConfig( #using class tuple to get o/p datatype as PrepareCallbacksConfig
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )
        return prepare_callbacks_config

In [5]:
import os
import urllib.request as request #to work on url (while downloading)
from zipfile import ZipFile # for zip
import tensorflow as tf
import time

#creating callback component
#Callback is a method where we save the model from time to time so that in case our training is stopped by some reason, then we can start with latest model.
#It also maintains tensorflow logs so we get to know the status of training, accuracy and loss status, etc
#Callback examples are checkpoint callbacks, early stopping callbacks, tensorboard logs callbacks, etc.
#Checkpoint callback is similar to autosave after certain iterations/time steps in CFD workflow, whereas early stopping is similar to convergence achieved within certain threshold, and, tensorboard logs is like logfiles of .sim

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig): #takes input in local config variable with i/p datatype as PrepareCallbacksConfig
        self.config = config

    # single _ is used as a nomenclature for hidden method.
    @property #property decorator
    def _create_tb_callbacks(self):
        #creating directories with timestamp
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S") #timestamp use for creating unique directories. dir name will be Year-month-date-hr-min-sec
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
            )        
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True #save only best weights
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks, #for this fun we are not using (), bcoz we have decorated this function with @property decorator.
            self._create_ckpt_callbacks
        ]
    

In [6]:
#pipeline section
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config = prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

In [6]:
import os
os.path.dirname("x/y/z.txt")
# Above method returns only dir name. Here z.txt is file in dir y. dirname fun returns only dirname/dirpath.

'x/y'

In [8]:
import time
time.strftime("%Y-%m-%d-%H-%M-%S")

'2023-05-14-11-45-01'